[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/HyperCoast/blob/main/docs/workshops/pace.ipynb)

# Working with NASA PACE data in HyperCoast

This notebook demonstrates how to visualize and analyze Plankton, Aerosol, Cloud, ocean Ecosystem ([PACE](https://pace.gsfc.nasa.gov/)) data interactively with HyperCoast.

## Environment setup

Uncomment and run the following cell to install the required packages.

In [ ]:
# %pip install "hypercoast[extra]"

Import library.

In [ ]:
import hypercoast

## Search for PACE data

To download and access the data, you will need to create an Earthdata login. You can register for an account at [urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov). Once you have an account, run the following cell and enter your NASA Earthdata login credentials.

In [ ]:
hypercoast.nasa_earth_login(persist=True)

### Search data programmatically

To search for PACE data programmatically, specify the bounding box and time range of interest. Set `count=-1` to return all results or set `count=10` to return the first 10 results.

In [ ]:
results, gdf = hypercoast.search_pace(
    bounding_box=(-83, 25, -81, 28),
    temporal=("2024-07-30", "2024-08-15"),
    short_name="PACE_OCI_L2_AOP_NRT",
    count=10,
    return_gdf=True,
)

Plot the footprints of the returned datasets on a map.

In [ ]:
gdf.explore()

Download the first dataset from the search results. Note that the download may take some time.

In [ ]:
hypercoast.download_pace(results[:1], out_dir="data")

### Search data interactively

To search for PACE data interactively, pan and zoom to the area of interest. Specify the time range of interest from the search dialog, then click on the Search button.

In [ ]:
m = hypercoast.Map(center=[30.0262, -90.1345], zoom=8)
m.search_pace(default_dataset="PACE_OCI_L2_AOP_NRT")
m

By default, the `search_pace` method search for the `PACE_OCI_L2_AOP_NRT` dataset, but you can specify the dataset name by setting the `default_dataset` parameter, such as `PACE_OCI_L2_BGC_NRT`. For more information about the available datasets, see the [PACE Data Products](https://pace.oceansciences.org/data_table.htm) page.

![image](https://github.com/user-attachments/assets/2c45ad43-c405-402a-92e8-42f497730fbb)

Uncomment the following cell to display the GeoDataFrame of the search results.

In [ ]:
# m._NASA_DATA_GDF.head()

Similarly, you can download the first dataset from the search results by uncommenting the following cell.

In [ ]:
# hypercoast.download_pace(results[:1], out_dir="data")

## Read PACE data

Let's download a sample PACE Apparent Optical Properties ([AOP](https://pace.oceansciences.org/pace_eq_aop.htm)) dataset for the demonstration.

In [ ]:
results = hypercoast.search_pace(
    bounding_box=(-83, 25, -81, 28),
    temporal=("2024-07-30", "2024-08-15"),
    short_name="PACE_OCI_L2_AOP_NRT",
    count=1,
)

In [ ]:
hypercoast.download_pace(results[:1], out_dir="data")

Let's make a scatter plot of the pixel locations so we can see the irregular spacing.

In [ ]:
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
plot = hypercoast.view_pace_pixel_locations(filepath, step=20)

![image](https://github.com/user-attachments/assets/91a0edd8-3c18-48a1-af75-ca0fc42bace1)

Load the dataset as a `xarray.Dataset` object.

In [ ]:
dataset = hypercoast.read_pace(filepath)
# dataset

![image](https://github.com/user-attachments/assets/56b1fae3-9acf-4ee1-8dc9-7f6784bedf88)

## Visualize PACE data

Visualize selected bands of the dataset.

In [ ]:
hypercoast.viz_pace(dataset, wavelengths=[500, 510, 520, 530], ncols=2)

![image](https://github.com/user-attachments/assets/f02ee287-06f5-45c0-a7d6-1c8ace3e8321)

Add custom projection and administrative boundaries to the map. The default projection is `PlateCarree`. You can specify a custom projection by setting the `crs` parameter. For more information about the available projections, see the [cartopy projection](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html) page.

In [ ]:
hypercoast.viz_pace(dataset, wavelengths=[500, 510, 520, 530], ncols=2, crs="default")

![image](https://github.com/user-attachments/assets/d94e5fc4-43c2-4bec-9432-2d0ff15a72b8)

## Plot spectral signatures

Plot a spectral signature of a pixel.

In [ ]:
latitude = 29.9307
longitude = -87.9106
hypercoast.extract_pace(dataset, latitude, longitude, return_plot=True)

![image](https://github.com/user-attachments/assets/4ca54e13-3c68-4dba-b89b-69af0180114e)

Plot spectral signatures of multiple pixels.

In [ ]:
latitude = (29.49, 29.50)
longitude = (-88.10, -88.00)
hypercoast.filter_pace(dataset, latitude, longitude, return_plot=True)

![image](https://github.com/user-attachments/assets/fdc6aebe-f265-4a23-9608-198253795c03)

## Interactive visualization

### Single-band visualization

Visualize a selected band of the dataset interactively use the `add_pace` method and speccify the `wavelengths` parameter.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
wavelengths = [450]
m.add_pace(dataset, wavelengths, colormap="jet", vmin=0, vmax=0.02, layer_name="PACE")
m.add_colormap(cmap="jet", vmin=0, vmax=0.02, label="Reflectance")
m.add("spectral")
m.set_center(-80.7382, 26.5295, zoom=6)
m

![image](https://github.com/user-attachments/assets/6c8fd406-4655-4e2e-9270-ae3c5f484121)

### Multi-band visualization

Select three spectral bands to visualize as an RGB image.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
wavelengths = [450, 550, 650]
m.add_pace(
    dataset, wavelengths, indexes=[3, 2, 1], vmin=0, vmax=0.02, layer_name="PACE"
)
m.add("spectral")
m.set_center(-80.7382, 26.5295, zoom=6)
m

![image](https://github.com/user-attachments/assets/a5b28084-f958-437c-b492-376a15451baa)

### Change band combination

In [ ]:
m

In [ ]:
# hypercoast.nasa_earth_login()

# short_name = "PACE_OCI_L2_BGC_NRT"
# results, gdf = hypercoast.search_nasa_data(
#     short_name=short_name,
#     bbox=(-90.5642, 29.9749, -89.7143, 30.42),
#     temporal=("2024-06-15", "2024-06-16"),
#     return_gdf=True
#     )
# hypercoast.download_nasa_data(results, out_dir="bgc")

Alternatively, use the following code block to download a sample dataset from [here](https://github.com/opengeos/datasets/releases/tag/hypercoast).

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/PACE_OCI.20240615T182549.L2.OC_BGC.V1_0_0.NRT.nc"
filepath = "data/PACE_OCI.20240615T182549.L2.OC_BGC.V1_0_0.NRT.nc"
hypercoast.download_file(url, filepath)

Load the downloaded dataset as an `xarray.Dataset`:

In [ ]:
dataset = hypercoast.read_pace_bgc(filepath)

Let's inspect the data variables contained in the dataset:

In [ ]:
dataset.variables

We can see that the dataset contains the following variables:
- [Chlorophyll Concentration](https://www.earthdata.nasa.gov/apt/documents/chlor-a/v1.0)
- [Phytoplankton Carbon](https://oceancolor.gsfc.nasa.gov/resources/atbd/cphyt/)
- [Particulate Organic Carbon](https://oceancolor.gsfc.nasa.gov/resources/atbd/poc/)

Transform the xarray dataset into gridded data. 

Plot the Chlorophyll Concentration.

In [ ]:
chlor_a = hypercoast.grid_pace_bgc(dataset, variable="chlor_a", method="linear")
chlor_a.plot(vmin=0, vmax=20, cmap="jet", size=6)

Plot the Phytoplankton Carbon.

In [ ]:
carbon_phyto = hypercoast.grid_pace_bgc(
    dataset, variable="carbon_phyto", method="linear"
)
carbon_phyto.plot(vmin=0, vmax=120, cmap="jet", size=6)

Particulate Organic Carbon.

In [ ]:
poc = hypercoast.grid_pace_bgc(dataset, variable="poc", method="linear")
poc.plot(vmin=0, vmax=1000, cmap="jet")

Plot the data on an interactive map.

In [ ]:
m = hypercoast.Map()
m.add_basemap("Hybrid")
m.add_raster(chlor_a, layer_name="Chlorophyll-a", colormap="jet", vmin=0, vmax=20)
m.add_raster(
    carbon_phyto, layer_name="Phytoplankton Carbon", colormap="plasma", vmin=0, vmax=120
)
m.add_raster(
    poc, layer_name="Particulate Organic Carbon", colormap="coolwarm", vmin=0, vmax=1000
)
m.add_layer_manager()

m.add_colormap(cmap="jet", vmin=0, vmax=20, label="Chlorophyll-a (mg/m3)")
m.add_colormap(cmap="plasma", vmin=0, vmax=120, label="Phytoplankton Carbon (mg/m3)")
m.add_colormap(
    cmap="coolwarm", vmin=0, vmax=1000, label="Particulate Organic Carbon (mg/m3)"
)
m

![](https://i.imgur.com/AEccc5k.gif)



To download PACE data, you will need to create an Earthdata login. You can register for an account at [urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov). Once you have an account, you can uncomment the code below to search and download the data.

In [ ]:
# hypercoast.nasa_earth_login()
# temporal = ("2024-06-01", "2024-07-01")
# results= hypercoast.search_pace_chla(temporal=temporal)
# hypercoast.download_nasa_data(results, "chla")

Alternatively, you can download some sample data from [here](https://github.com/opengeos/datasets/releases/tag/hypercoast).

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/pace_chla.zip"
hypercoast.download_file(url)

The downloaded zip file is automatically extracted and saved in the `chla` directory, which contains 17 daily files of chlorophyll-a concentration data in the netCDF format. The date range of the data is from 2024-06-01 to 2024-06-17.

In [ ]:
files = "chla/*nc"

Load all the data files in the `chla` directory as an xarray DataArray

In [ ]:
array = hypercoast.read_pace_chla(files)
array

Select a date and visualize the chlorophyll-a concentration data with Matplotlib.

In [ ]:
hypercoast.viz_pace_chla(array, date="2024-06-01", cmap="jet", size=6)

If the date is not specified, the data are averaged over the entire time range.

In [ ]:
hypercoast.viz_pace_chla(array, cmap="jet", size=6)

To visualize the data interactively, we can select either a single date or aggregate the data over a time range. 

First, let's select a single date from the data array:

In [ ]:
single_array = array.sel(date="2024-06-01")
single_array

Convert the data array to an image that can be displayed on an interactive map.

In [ ]:
single_image = hypercoast.pace_chla_to_image(single_array)

Create an interactive map and display the image on the map.

In [ ]:
m = hypercoast.Map(center=[40, -100], zoom=4)
m.add_basemap("Hybrid")
m.add_raster(
    single_image,
    cmap="jet",
    vmin=-1,
    vmax=2,
    layer_name="Chlorophyll a",
    zoom_to_layer=False,
)
label = "Chlorophyll Concentration [lg(lg(mg m^-3))]"
m.add_colormap(cmap="jet", vmin=-1, vmax=2, label=label)
m

![](https://i.imgur.com/AQr9wUP.png)

The daily image does not have a global coverage. To visualize the data globally, we can aggregate the data over a time range.

In [ ]:
mean_array = array.mean(dim="date")

Convert the aggregated data array to an image that can be displayed on an interactive map.

In [ ]:
image = hypercoast.pace_chla_to_image(mean_array)

Create an interactive map and display the image on the map.

In [ ]:
m = hypercoast.Map(center=[40, -100], zoom=4)
m.add_basemap("Hybrid")
m.add_raster(
    image, cmap="jet", vmin=-1, vmax=2, layer_name="Chlorophyll a", zoom_to_layer=False
)
label = "Chlorophyll Concentration [lg(lg(mg m^-3))]"
m.add_colormap(cmap="jet", vmin=-1, vmax=2, label=label)
m

![](https://i.imgur.com/6hP6OFD.png)

In [ ]:
import hypercoast
import pandas as pd

Download a sample filed dataset.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/pace_sample_points.csv"
data = pd.read_csv(url)
data.head()

Download PACE data.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
filepath = "data/PACE_OCI.20240730T181157.L2.OC_AOP.V2_0.NRT.nc"
hypercoast.download_file(url, filepath)

Read the PACE dataset.

In [ ]:
dataset = hypercoast.read_pace(filepath)

Run the following cell to show the map. Click on the markers to see the spectral data.

In [ ]:
m = hypercoast.Map(center=[27.235094, -87.791748], zoom=6)

m.add_basemap("Hybrid")
wavelengths = [450, 550, 650]
m.add_pace(
    dataset, wavelengths, indexes=[3, 2, 1], vmin=0, vmax=0.02, layer_name="PACE"
)
m.add("spectral")

m.add_field_data(
    data,
    x_col="wavelength",
    y_col_prefix="(",
    x_label="Wavelength (nm)",
    y_label="Reflectance",
    use_marker_cluster=True,
)
m.set_center(-87.791748, 27.235094, zoom=6)
m

![](https://i.imgur.com/Q8pBQXg.png)